#### Three ways to deal with categorical variables

1. Drop columns with categorical variables from the dataset - works well only if those features were not important
2. Assign each category a number - works well if the categories are ordinal
3. Create a one-hot encoding of the categories in separate columns - works well a small number (< 15) of nominal categories 

<br>

In general, one-hot encoding (Approach 3) will typically perform best, and dropping the categorical columns (Approach 1) typically performs worst, but it varies on a case-by-case basis.

In [1]:
import pandas as pd

data = pd.read_csv('../data/melb_data.csv')

In [2]:
data.shape

(13580, 21)

In [3]:
from sklearn.model_selection import train_test_split

y = data.Price
X = data.drop(['Price'], axis=1)

X_train_full, X_valid_full, y_train, y_valid = \
train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [4]:
X_train_full.head()

,Suburb,Address,Rooms,Type,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
12167,St Kilda,11/22 Charnwood Cr,1,u,S,hockingstuart,29/07/2017,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,Port Phillip,-37.85984,144.9867,Southern Metropolitan,13240.0
6524,Williamstown,18 James St,2,h,SA,Hunter,17/09/2016,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,Hobsons Bay,-37.85800,144.9005,Western Metropolitan,6380.0
8413,Sunshine,10 Dundalk St,3,h,S,Barry,8/04/2017,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,Brimbank,-37.79880,144.8220,Western Metropolitan,3755.0
2919,Glenroy,1/2 Prospect St,3,u,SP,Brad,18/06/2016,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,Moreland,-37.70830,144.9158,Northern Metropolitan,8870.0
6043,Sunshine North,35 Furlong Rd,3,h,S,First,22/05/2016,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,Brimbank,-37.76230,144.8272,Western Metropolitan,4217.0


In [5]:
# Drop columns with missing values (simplest approach)

cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()]

X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

/Users/jessemurray/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
# Select categorical columns with relatively low cardinality
low_cardinality_cols = [cname for cname in X_train_full.columns if \
                        X_train_full[cname].nunique() < 10 and \
                        X_train_full[cname].dtype == 'object']

In [7]:
numerical_cols = [cname for cname in X_train_full.columns if \
              X_train_full[cname].dtype in ['float64', 'int64']]

In [8]:
my_cols = low_cardinality_cols + numerical_cols

In [9]:
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [10]:
X_train

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.98670,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.90050,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.82200,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.91580,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.82720,4217.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13123,h,SP,Northern Metropolitan,3,5.2,3056.0,3.0,1.0,212.0,-37.77695,144.95785,11918.0
3264,h,S,Eastern Metropolitan,3,10.5,3081.0,3.0,1.0,748.0,-37.74160,145.04810,2947.0
9845,h,PI,Northern Metropolitan,4,6.7,3058.0,4.0,2.0,441.0,-37.73572,144.97256,11204.0
10799,h,S,Northern Metropolitan,3,12.0,3073.0,3.0,1.0,606.0,-37.72057,145.02615,21650.0


In [11]:
print('Categorical variables:')
object_cols = low_cardinality_cols
print(object_cols)

Categorical variables:
['Type', 'Method', 'Regionname']


In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [13]:
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.get_params()

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}

In [14]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])
print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
175703.48185157913


In [19]:
from sklearn.preprocessing import LabelEncoder

label_X_train = X_train.reset_index(drop=True).copy()
label_X_valid = X_valid.copy()

# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()

for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])
    print(label_encoder.classes_)
    print()

['h' 't' 'u']

['PI' 'S' 'SA' 'SP' 'VB']

['Eastern Metropolitan' 'Eastern Victoria' 'Northern Metropolitan'
 'Northern Victoria' 'South-Eastern Metropolitan' 'Southern Metropolitan'
 'Western Metropolitan' 'Western Victoria']



`LabelEncoder` seems to assign label encoders (numbers 0 through n_classes - 1) based on alphabetical order, which is essentially random.

In [16]:
X_train.reset_index(drop=True)

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
0,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.98670,13240.0
1,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.90050,6380.0
2,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.82200,3755.0
3,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.91580,8870.0
4,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.82720,4217.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10859,h,SP,Northern Metropolitan,3,5.2,3056.0,3.0,1.0,212.0,-37.77695,144.95785,11918.0
10860,h,S,Eastern Metropolitan,3,10.5,3081.0,3.0,1.0,748.0,-37.74160,145.04810,2947.0
10861,h,PI,Northern Metropolitan,4,6.7,3058.0,4.0,2.0,441.0,-37.73572,144.97256,11204.0
10862,h,S,Northern Metropolitan,3,12.0,3073.0,3.0,1.0,606.0,-37.72057,145.02615,21650.0


In [17]:
label_X_train

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
0,2,1,5,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.98670,13240.0
1,0,2,6,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.90050,6380.0
2,0,1,6,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.82200,3755.0
3,2,3,2,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.91580,8870.0
4,0,1,6,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.82720,4217.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10859,0,3,2,3,5.2,3056.0,3.0,1.0,212.0,-37.77695,144.95785,11918.0
10860,0,1,0,3,10.5,3081.0,3.0,1.0,748.0,-37.74160,145.04810,2947.0
10861,0,0,2,4,6.7,3058.0,4.0,2.0,441.0,-37.73572,144.97256,11204.0
10862,0,1,2,3,12.0,3073.0,3.0,1.0,606.0,-37.72057,145.02615,21650.0


If there are categorical columns with with values that don't appear in the training data but appear in the validation data, this will raise an error. Because, the label encoder is fitted to the training data such that it creates an integer-valued label for each unique value in the categorical columns of the training data. 

In [50]:
# Set subtraction goes as follows: in the left but not in the right
print(set(['a', 'b']) - set(['b', 'c']))
print(set(['a', 'b']).difference(set(['b', 'c'])))
print('(Equivalent)')

{'a'}
{'a'}
(Equivalent)


In [51]:
print(set(['b', 'c']).issubset(set(['a', 'b', 'c'])))
print(set(['b', 'c']) <= set(['a', 'b', 'c']))
print('(Equivalent)')

True
True
(Equivalent)


In [45]:
# All categorical columns
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely label encoded
# X_train[col].unique() would also work here. 
# Also it may be better to use <= instead of == because we only want to prevent against
# X_valid having values not in X_train. It's okay if X_train has values not in X_valid. 
good_label_cols = [col for col in object_cols if set(X_valid[col]) == set(X_train[col])]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be label encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

Categorical columns that will be label encoded: ['Type', 'Method', 'Regionname']

Categorical columns that will be dropped from the dataset: []


We don't have this problem in this dataset with this particular train-test split.

In [20]:
print("MAE from Approach 2 (Label Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Label Encoding):
165936.40548390493


In [21]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))


* We set `handle_unknown='ignore'` to avoid errors when the validation data contains classes that aren't represented in the training data.
* Setting `sparse=False` ensures that the encoded columns are returned as a numpy array (instead of a sparse matrix).

There is one column for each class

In [22]:
OH_cols_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10860,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10861,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10862,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
166089.4893009678


In [63]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))

d = dict(zip(object_cols, object_nunique))

print(object_cols)
print()
print(d)
print()
print(object_nunique)
print()


# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

['Type', 'Method', 'Regionname']

{'Type': 3, 'Method': 5, 'Regionname': 8}

[3, 5, 8]



[('Type', 3), ('Method', 5), ('Regionname', 8)]

The output above shows, for each column with categorical data, the number of unique values in the column. In other words, the *cardinality* of each categorical variable. 

Drop high cardinality columns

In [69]:
# Columns that will be one-hot encoded
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

# Columns that will be dropped from the dataset
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)

Categorical columns that will be one-hot encoded: ['Type', 'Method', 'Regionname']

Categorical columns that will be dropped from the dataset: []
